In [1]:
from gensim import corpora, models, similarities
import gensim

In [2]:
import json 
import gzip 
#from StringIO import StringIO
import urllib.request
import requests, zipfile, io
import re

def parse(url): 
    if re.match("^http", url):
        r = requests.get(url)
        g = gzip.open(io.BytesIO(r.content))
    else:
        g = gzip.open(url, 'r') 
    for l in g:
        yield eval(l)

import json 
import gzip 
def parse(url): 
    g = gzip.open(url, 'r') 
    for l in g: 
        yield eval(l)

In [3]:
import gensim
import nltk
stoplist = nltk.corpus.stopwords.words('english')
lemma = nltk.stem.wordnet.WordNetLemmatizer()

def myReviews(review_files_urls, catFilter = None, asin = None, concatenate = False, max_count = None):
    preprocess_reviewText = []
    for url in review_files_urls:
        print("Processing url: " + url)
        review_item_iterator = iter(parse(url))
        it = 0
        conc_count = 0
        for review_item in review_item_iterator:
            it += 1
            if (max_count is not None) and (it > max_count):
                return
        #for it in range(1000000):
            if it%100000==0:
                print("it="+str(it))

            reviewText = review_item["reviewText"]
            label = review_item["asin"] + "_" + review_item["reviewerID"] 
            preprocess_reviewText_i = gensim.utils.simple_preprocess(reviewText)
            preprocess_reviewText_i = [lemma.lemmatize(word) for word in preprocess_reviewText_i if word not in stoplist]


            #yield gensim.models.doc2vec.LabeledSentence(words=preprocess_reviewText, tags=[label])
            if (catFilter == None or (catFilter in review_item["categories"])) and (asin == None or (asin == review_item["asin"])):
                if concatenate==False:
                    yield preprocess_reviewText_i
                else:
                    conc_count += 1
                    preprocess_reviewText.extend(preprocess_reviewText_i)
                    print("Concatenated count: "+str(conc_count))
    if concatenate == True:
        print("Return the final text with length: "+str(len(preprocess_reviewText)))
        print(preprocess_reviewText)
        yield preprocess_reviewText

#base_url = "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/"
base_url = "/media/mister/ntfs/Rainer/y2buy/download/amazon_reviews_96_14/"

from bs4 import BeautifulSoup
import urllib.request


html = urllib.request.urlopen("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/")
soup = BeautifulSoup(html, 'html.parser')
#print(soup.prettify())
all_html_links = soup.find_all('a')
review_file_names = [links["href"] for links in all_html_links if re.match("^review.*[^_5|_10].json.gz$", links["href"])]

In [4]:
from os import listdir
from os.path import isfile, join
import re
base_url = "/media/mister/ntfs/Rainer/y2buy/download/amazon_reviews_96_14/"
onlyfiles = [f for f in listdir(base_url) if isfile(join(base_url, f))]
review_file_names = [file for file in onlyfiles if re.match("^review.*[^_5|_10].json.gz$", file)]

FileNotFoundError: [Errno 2] No such file or directory: '/media/mister/ntfs/Rainer/y2buy/download/amazon_reviews_96_14/'

for links in all_html_links:
    if re.match("^review.*[^_5|_10].json.gz$", links["href"]):
        print(links["href"])

In [ ]:


review_file_names= ["reviews_Books_10.json.gz" if file_name =="reviews_Books.json.gz" else file_name for file_name in review_file_names]
print(review_file_names)
review_file_urls = [base_url + review_file_name for review_file_name in review_file_names]
#print(review_file_urls)

In [5]:
  
import os
dictionaryFileName = '../../download/word2vec/reviews_all.dict'
if os.path.isfile(dictionaryFileName)==False:
    #dictionary = corpora.Dictionary(myReviews("Cell Phones"))
    dictionary = corpora.Dictionary()
    for review_file_url in review_file_urls:
        dictionary.add_documents(myReviews([review_file_url],max_count = 1000000))
        from six import iteritems
        once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
        dictionary.filter_tokens(once_ids)  # remove stop words and words that appear only once
        dictionary.compactify()  # remove gaps in id sequence after words that were removed
        print(dictionary)
    dictionary.save(dictionaryFileName)  # store the dictionary, for future reference
else:
    dictionary = corpora.Dictionary.load(dictionaryFileName)




    
import os
dictionaryFileName = '../../download/word2vec/reviews_all.dict'
if os.path.isfile(dictionaryFileName)==False:
    #dictionary = corpora.Dictionary(myReviews("Cell Phones"))
    dictionary = corpora.Dictionary(myReviews(review_file_urls))
    from six import iteritems
    once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
    dictionary.filter_tokens(once_ids)  # remove stop words and words that appear only once
    dictionary.compactify()  # remove gaps in id sequence after words that were removed
    dictionary.save(dictionaryFileName)  # store the dictionary, for future reference
else:
    dictionary = corpora.Dictionary.load(dictionaryFileName)

print(dictionary)

In [6]:
def myCorpus(review_file_urls, catFilter = None, asin = None, concatenate = False, max_count = None):
    for review in myReviews(review_file_urls,catFilter,asin,concatenate,max_count):
    # assume there's one document per line, tokens separated by whitespace
        yield dictionary.doc2bow(review)

In [7]:
import os
tfidfFileName ='../../download/word2vec/reviews_all_corpus.tfidf'
if os.path.isfile(tfidfFileName)==False:
    #tfidf = models.TfidfModel(myCorpus("Cell Phones"))
    tfidf = models.TfidfModel(myCorpus(review_file_urls,max_count = 1000000))
    tfidf.save(tfidfFileName)
else:
    tfidf = models.TfidfModel.load(tfidfFileName)

print(tfidf)

TfidfModel(num_docs=1093393, num_nnz=41632789)


In [8]:
import pickle

#tfidf_document_FileName ='../../download/word2vec/reviews_cell_phones_review.tfidf'
tfidf_document_FileName ='../../download/word2vec/reviews_cell_phones.corpus'
if os.path.isfile(tfidf_document_FileName)==False:
    document = next(myCorpus(review_file_urls = ['../../download/word2vec/reviews_Cell_Phones_and_Accessories_w_Cat.json.gz'], catFilter="Cell Phones",concatenate=True))
    document_tfidf = tfidf[document]  
    pickle.dump(document, open(tfidf_document_FileName, 'wb'))
else:
    document = pickle.load( open( tfidf_document_FileName, "rb" ) )
    document_tfidf = tfidf[document]  
    
document_dict = dict((tokenid, count) for tokenid, count in document)
    
    


In [9]:
import gensim
model = gensim.models.Word2Vec.load('../../download/word2vec/myamazonmodel.word2vec')
#model = gensim.models.KeyedVectors.load_word2vec_format('../../download/word2vec/GoogleNews-vectors-negative300.bin', binary=True, limit=400000) 

In [10]:
import numpy as np
import pandas as pd
tbl =[[dictionary.get(tokenid), document_dict[tokenid], tfidf_value ,model.wv[dictionary.get(tokenid)] ] for (tokenid,tfidf_value) in document_tfidf if tfidf_value > 0.02 and dictionary.get(tokenid) in model.wv]

df = pd.DataFrame(data=tbl,
                  columns=["token","term_freq","tfidf", "word_vector"]
                  )
df = df.sort_values('tfidf', ascending=False)
df.set_index("token")
print(df)

         token  term_freq     tfidf  \
94       phone     424902  0.836701   
135        sim      17723  0.096396   
114    android      21694  0.094238   
101     screen      51072  0.089932   
88        call      36115  0.089226   
128       apps      22849  0.088002   
102     mobile      20957  0.075298   
103      nokia      15863  0.074846   
138   unlocked      13680  0.072165   
96     battery      48257  0.070360   
112     iphone      21064  0.067877   
67        text      17968  0.058434   
27        good      63078  0.056743   
35        like      55524  0.055823   
106    samsung      18165  0.053266   
58         use      59604  0.051866   
17         one      66062  0.051633   
2          get      52578  0.051574   
119    service      23268  0.049913   
93        cell      16293  0.049780   
19        card      27391  0.049188   
78        time      45455  0.048835   
76         day      30518  0.048210   
142   tracfone       7189  0.047962   
54         new      32494

In [11]:
import nltk
from nltk.cluster.kmeans import KMeansClusterer
nc = 5
data = df.loc[:,"word_vector"]
#print(data)
kclusterer = KMeansClusterer(nc, distance=nltk.cluster.util.cosine_distance, avoid_empty_clusters=True) #repeats=30,
assigned_clusters = kclusterer.cluster(data, assign_clusters=True)
#print(assigned_clusters)
means = np.asarray( kclusterer.means())


In [12]:
df["cluster"] = pd.Series(assigned_clusters, index=df.index)

print(df.shape)
print(df)

(145, 5)
         token  term_freq     tfidf  \
94       phone     424902  0.836701   
135        sim      17723  0.096396   
114    android      21694  0.094238   
101     screen      51072  0.089932   
88        call      36115  0.089226   
128       apps      22849  0.088002   
102     mobile      20957  0.075298   
103      nokia      15863  0.074846   
138   unlocked      13680  0.072165   
96     battery      48257  0.070360   
112     iphone      21064  0.067877   
67        text      17968  0.058434   
27        good      63078  0.056743   
35        like      55524  0.055823   
106    samsung      18165  0.053266   
58         use      59604  0.051866   
17         one      66062  0.051633   
2          get      52578  0.051574   
119    service      23268  0.049913   
93        cell      16293  0.049780   
19        card      27391  0.049188   
78        time      45455  0.048835   
76         day      30518  0.048210   
142   tracfone       7189  0.047962   
54         new  

In [13]:
df["centroid"] = pd.Series( means[assigned_clusters].tolist(), index=df.index)

print(df.shape)
print(df)


(145, 6)
         token  term_freq     tfidf  \
94       phone     424902  0.836701   
135        sim      17723  0.096396   
114    android      21694  0.094238   
101     screen      51072  0.089932   
88        call      36115  0.089226   
128       apps      22849  0.088002   
102     mobile      20957  0.075298   
103      nokia      15863  0.074846   
138   unlocked      13680  0.072165   
96     battery      48257  0.070360   
112     iphone      21064  0.067877   
67        text      17968  0.058434   
27        good      63078  0.056743   
35        like      55524  0.055823   
106    samsung      18165  0.053266   
58         use      59604  0.051866   
17         one      66062  0.051633   
2          get      52578  0.051574   
119    service      23268  0.049913   
93        cell      16293  0.049780   
19        card      27391  0.049188   
78        time      45455  0.048835   
76         day      30518  0.048210   
142   tracfone       7189  0.047962   
54         new  

In [14]:
df["centroid_distance"] = df.apply(lambda row:nltk.cluster.util.cosine_distance(row['word_vector'], row['centroid']), axis=1)

print(df.shape)
print(df)

(145, 7)
         token  term_freq     tfidf  \
94       phone     424902  0.836701   
135        sim      17723  0.096396   
114    android      21694  0.094238   
101     screen      51072  0.089932   
88        call      36115  0.089226   
128       apps      22849  0.088002   
102     mobile      20957  0.075298   
103      nokia      15863  0.074846   
138   unlocked      13680  0.072165   
96     battery      48257  0.070360   
112     iphone      21064  0.067877   
67        text      17968  0.058434   
27        good      63078  0.056743   
35        like      55524  0.055823   
106    samsung      18165  0.053266   
58         use      59604  0.051866   
17         one      66062  0.051633   
2          get      52578  0.051574   
119    service      23268  0.049913   
93        cell      16293  0.049780   
19        card      27391  0.049188   
78        time      45455  0.048835   
76         day      30518  0.048210   
142   tracfone       7189  0.047962   
54         new  

In [15]:
writer = pd.ExcelWriter('clustered_smartphone_features.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()

print(tfidf)